<a href="https://colab.research.google.com/github/wojter/road-users-detection/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Config

In [ ]:
import os
from pathlib import Path
import tensorflow as tf
import re

In [ ]:
# local or colab
drive_mode = True

# images scaled and packed in archive of var name @IMAGES_ARCHIVE
archived_images = True

In [ ]:
LABELS = [
    {'name':'bike', 'id':1}, 
    {'name':'scooter', 'id':2},
    {'name':'rolls', 'id':3},
    {'name':'pedestrian', 'id':4},
    {'name':'uto', 'id':5}
]


MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
#'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
#'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'

IMAGE_WIDTH = 640
IMAGE_HEIGHT = 640

BATCH_SIZE = 8
NUM_STEPS = 10000
NUM_EVAL_STEPS = 200
CHECKPOINT_STEPS = 500

In [ ]:
# folders

# paths to images on instance 
IMAGES = os.path.join('images')
IMAGES_TEST = os.path.join('images', 'test')
IMAGES_TRAIN = os.path.join('images', 'train')

TENSORFLOW_MODELS = os.path.join('external', 'tensorflow_models')
DOWNLOADED_MODELS = os.path.join('external', 'downloaded_models')
TF_RECORDS = os.path.join('external', 'tf_records')

DRIVE_NAME = os.path.join('drive')
GOOGLE_DRIVE = os.path.join(DRIVE_NAME, 'MyDrive')
GOOGLE_DRIVE_COLAB = os.path.join(GOOGLE_DRIVE, 'COLAB')
GOOGLE_DRIVE_IMAGES = os.path.join(GOOGLE_DRIVE, 'projects','images')
IMAGES_ARCHIVE = os.path.join('archive.tar.gz')
GOOGLE_DRIVE_IMG_ARCHIVE = os.path.join(GOOGLE_DRIVE, 'projects', IMAGES_ARCHIVE)

# files
LABEL_FILE = os.path.join('label_map.txt')
TF_RECORD_FILE = os.path.join(TF_RECORDS, 'generate_tfrecord.py')
TRAIN_RECORD = 'train.record'
TEST_RECORD = 'test.record'
VAL_RECORD = 'val.record'
PIPELINE_CONFIG_PATH = 'model_config.config'

FINE_TUNE_CHECKPOINT = os.path.join(DOWNLOADED_MODELS, MODEL_NAME, 'checkpoint', 'ckpt-0')
MODEL_DIR = os.path.join(GOOGLE_DRIVE, 'training') if drive_mode else os.path.join('training')
BASE_CONFIG_PATH = os.path.join(TENSORFLOW_MODELS, 'research', 'object_detection', 'configs', 'tf2', f'{MODEL_NAME}.config')


# 2. Download and prepare required soft

In [ ]:
if drive_mode:
  from google.colab import drive
  drive.mount(DRIVE_NAME)

In [ ]:
# get from github
if not os.path.exists(TENSORFLOW_MODELS):
  !git clone --depth 1 https://github.com/tensorflow/models {TENSORFLOW_MODELS}
if not os.path.exists(TF_RECORDS):
  !git clone --depth 1  https://github.com/nicknochnack/GenerateTFRecord {TF_RECORDS}

In [ ]:
if os.system(f" \
cd { TENSORFLOW_MODELS }/research; \
protoc object_detection/protos/*.proto --python_out=.; \
cp object_detection/packages/tf2/setup.py .; \
python -m pip install .") != 0:
  raise Exception("Can't install Object Detection API")

In [ ]:
# run model builder test
!python {TENSORFLOW_MODELS}/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
!nvidia-smi
if tf.test.gpu_device_name() == '':
    raise Exception('GPU IS MISSING')

In [ ]:
tf.keras.backend.clear_session()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        assert tf.config.experimental.get_memory_growth(gpus[0])
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
        

# 3. Images

In [ ]:
# copy images
if drive_mode and archived_images:
  !cd drive/MyDrive/projects/ && ls
  !cp drive/MyDrive/projects/archive.tar.gz ./
elif drive_mode and not archived_images:
  !cp -r {GOOGLE_DRIVE_IMAGES} {IMAGES}

In [ ]:
# unpack images if archived
if drive_mode and archived_images:
  if os.path.exists(IMAGES_ARCHIVE) and not os.path.exists(IMAGES):
    !tar -zxvf {IMAGES_ARCHIVE}

# 4. Labels and records

In [ ]:
with open(LABEL_FILE, 'w') as f:
    for label in LABELS:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
# tf record file generate
!python {TF_RECORD_FILE} -x {IMAGES}/train -l {LABEL_FILE} -o {TRAIN_RECORD} 
!python {TF_RECORD_FILE} -x {IMAGES}/test -l {LABEL_FILE} -o {TEST_RECORD}
!python {TF_RECORD_FILE} -x {IMAGES}/val -l {LABEL_FILE} -o {VAL_RECORD}

# 5. Model from object detection API

In [ ]:
if Path(f"{DOWNLOADED_MODELS}/{MODEL_NAME}.tar.gz").is_file():
    print("Model already exists")
else:
    os.system('mkdir -p ' + DOWNLOADED_MODELS)
    os.system(f"cd {DOWNLOADED_MODELS}; wget http://download.tensorflow.org/models/object_detection/tf2/20200711/{MODEL_NAME}.tar.gz")
    os.system(f"cd {DOWNLOADED_MODELS}; tar -xf {MODEL_NAME}.tar.gz")

## 6. Set config file

In [ ]:
# edit configuration file 
with open(BASE_CONFIG_PATH) as f:
    config = f.read()
with open('model_config.config', 'w') as f:
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(LABEL_FILE), config)
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(FINE_TUNE_CHECKPOINT), config)
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(TRAIN_RECORD), config)
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(VAL_RECORD), config)
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(len(LABELS)), config)
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(BATCH_SIZE), config)
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(NUM_STEPS), config)
  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  config = re.sub('max_detections_per_class: [0-9]+',
                  'max_detections_per_class: {}'.format(100), config)
  config = re.sub('max_total_detections: [0-9]+',
                  'max_total_detections: {}'.format(100), config)
  f.write(config)

In [ ]:
%cat {PIPELINE_CONFIG_PATH}